In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV

In [17]:
import os
import pandas as pd
import numpy as np

# Accedemos a cada train y a cada test tanto de X como de Y, y los dejamos metidos en variables para el siguiente paso!! 
# Los separamos en base path y el resto de la ruta de acceso para tener el código mas legible. 
current_dir = os.getcwd()
BASE_PATH = os.path.join(current_dir, "../data/csv_model")
TRAIN_PATHS = [
    "X_train_con_outliers.csv",
    "X_train_sin_outliers.csv"
]

# Concatenamos el base path al resto de la ruta de acceso para tener el código completo dentro de la variable que usaremos despues. 
TRAIN_DATASETS = []
for path in TRAIN_PATHS:
    TRAIN_DATASETS.append(
        # pd.read_excel(BASE_PATH + "/" + path)
        pd.read_csv(f"{BASE_PATH}/{path}") 
        # pd.read_excel(os.path.join(BASE_PATH, path))
    )

# Los separamos en base path y el resto de la ruta de acceso para tener el código mas legible. 
TEST_PATHS = [
    "X_test_con_outliers.csv",
    "X_test_sin_outliers.csv"
]

# Concatenamos el base path al resto de la ruta de acceso para tener el código completo dentro de la variable que usaremos despues. 
TEST_DATASETS = []
for path in TEST_PATHS:
    TEST_DATASETS.append(
        pd.read_csv(f"{BASE_PATH}/{path}")
    )

# Concatenamos el base path al resto de la ruta de acceso para tener el código completo dentro de la variable que usaremos despues. 
y_train = pd.read_csv(f"{BASE_PATH}/y_train.csv")
y_test = pd.read_csv(f"{BASE_PATH}/y_test.csv")

#### Boosting with `XGBClassifier` and `GridSearchCV`

In [18]:
from xgboost import XGBRegressor
from sklearn.metrics import r2_score

# Le pasamos los train y los test al modelo , lo entrenamos y metemos los resultados del test en una lista para poder comparar y escoger el mejor. 

results = []

for index, dataset in enumerate(TRAIN_DATASETS):
    print(index)
    model = XGBRegressor(random_state = 77, device='cuda')
    
    model.fit(dataset, y_train)
    y_pred_train = model.predict(dataset)
    y_pred_test = model.predict(TEST_DATASETS[index])

    results.append(
        {
            "train": r2_score(y_train, y_pred_train),
            "test": r2_score(y_test, y_pred_test)
        }
    )

results

0
1


[{'train': 0.8018825054168701, 'test': 0.3218003511428833},
 {'train': 0.8018825054168701, 'test': 0.3218003511428833}]

In [19]:
# Optimización del modelo. 

best_dataset = 0 # Porque de results, el mayor acuracy en train y test.

# Ponemos los hyperparametros según la documentación.  

hyperparams = {
    'n_estimators': [100, 150, 200],  # Número de árboles (estimadores)
    'learning_rate': [0.01, 0.05, 0.1, 0.2],  # Tasa de aprendizaje
    'max_depth': [3, 5, 6, 7, 10],  # Profundidad máxima de los árboles
    'subsample': [0.7, 0.8, 0.9, 1.0],  # Fracción de muestras por árbol
    'colsample_bytree': [0.7, 0.8, 0.9, 1.0],  # Fracción de características por árbol
    'gamma': [0, 0.1, 0.2, 0.5],  # Parámetro de regularización
    'min_child_weight': [1, 5, 10],  # Peso mínimo de las muestras para dividir un nodo
}

from sklearn.model_selection import GridSearchCV 

model = XGBRegressor(random_state = 77) # Volvemos a seleccionar el modelo sin entrenar. 
grid = GridSearchCV(model, hyperparams, scoring = "r2") # Metemos en el gris los nuevos hyperparametros

In [20]:
grid.fit(TRAIN_DATASETS[best_dataset], y_train)

KeyboardInterrupt: 

In [24]:
grid.best_estimator_

AttributeError: 'GridSearchCV' object has no attribute 'best_estimator_'

In [23]:
# Predecimos con el nuevo modelo ya entrenado por los unevos parametros.  

final_model = grid.best_estimator_
y_pred_train = final_model.predict(TRAIN_DATASETS[best_dataset])
y_pred_test = final_model.predict(TEST_DATASETS[best_dataset])

AttributeError: 'GridSearchCV' object has no attribute 'best_estimator_'

In [22]:
score_train = r2_score(y_train, y_pred_train),
score_test = r2_score(y_test, y_pred_test)

print(f'Score train: {score_train}')
print(f'Score test: {score_test}')

Score train: (0.8018825054168701,)
Score test: 0.3218003511428833


In [ ]:
# import pickle

# with open("/Users/julian/Desktop/vs code/Boosting_algorithm_model_for_machine_learning_Project_18_Julian_Lopez/Boosting_algorithm_model_for_machine_learning_Project_18_Julian_Lopez/models/Boosting_with_XGBClassifier_and_GridSearchCV_model.sav", "wb") as file:
#     pickle.dump(final_model, file)

#### MAE

In [21]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

results = []

for index, dataset in enumerate(TRAIN_DATASETS):
    print(index)
    model = XGBRegressor(random_state=77, device='cuda')
    
    model.fit(dataset, y_train)
    y_pred_train = model.predict(dataset)
    y_pred_test = model.predict(TEST_DATASETS[index])

    results.append(
        {
            "train_mae": mean_absolute_error(y_train, y_pred_train),
            "test_mae": mean_absolute_error(y_test, y_pred_test)
        }
    )

results


0
1


[{'train_mae': 0.8743095993995667, 'test_mae': 1.6540964841842651},
 {'train_mae': 0.8743095993995667, 'test_mae': 1.6540964841842651}]